
Here is a tutorial about generating text using a SOTA inspired language generation model, distilgpt2. This model lighter in weight and faster in language generation than the original OpenAI GPT2. Using this tutorial, you can train a language generation model which can generate text for any subject in English. Here, we will generate movie reviews by fine-tuning distilgpt2 on a sample of IMDB movie reviews.


Click on the link below and a file will be downloaded containing IMDB sample dataset of 1000 samples

http://files.fast.ai/data/examples/imdb_sample.tgz

Upload this file in this colab notebook using the upload button on the top left 

In [ ]:
### Extract the csv file from the uploaded tgz file

import tarfile
with tarfile.open('imdb_sample.tgz', 'r:gz') as tar:
    tar.extractall()

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('imdb_sample/texts.csv')

In [ ]:
### This is how the CSV look like
data

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False
...,...,...,...
995,negative,There are many different versions of this one ...,True
996,positive,Once upon a time Hollywood produced live-actio...,True
997,negative,Wenders was great with Million $ Hotel.I don't...,True
998,negative,Although a film with Bruce Willis is always wo...,True


Let's get the number of samples

In [ ]:
data.shape

(1000, 3)

For Finetuning distilgpt2, we just need the text field

In [ ]:
texts = list(set(data['text']))

In [ ]:
len(texts)

1000

Store the reviews in a txt file where each line of txt file is a single review 

In [5]:
file_name = 'testing.txt'
# with open(file_name, 'w') as f:
#     f.write(" |EndOfText|\n".join(texts))

Now, let's come to Transformers by Huggingface, and unleash the Transformers (Autobots... just kidding)

In [1]:
!pip install transformers
!git clone https://github.com/huggingface/transformers.git

     |████████████████████████████████| 4.0 MB 27.3 MB/s 
     |████████████████████████████████| 895 kB 57.5 MB/s 
     |████████████████████████████████| 77 kB 8.9 MB/s 
     |████████████████████████████████| 6.6 MB 46.5 MB/s 
     |████████████████████████████████| 596 kB 48.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'transformers'...
remote: Enumerating objects: 109897, done.
remote: Total 109897 (delta 0), reused 0 (delta 0), pack-reused 109897
Receiving objects: 100% (109897/109897), 97.20 MiB | 15.26 MiB/s, done.
Resolving deltas: 100% (80162/80162), done.


In [11]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-h_5cd1jx
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-h_5cd1jx
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4030368 sha256=0039bc5f2d7fbee4458c411e4ee59e65ead526b40745d0eae20abdc6f2cf7607
  Stored in directory: /tmp/pip-ephem-wheel-cache-92nimnvi/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers


Make 2 directories. 

1) weights - for storing the weights of distilgpt2

2) tokenizer - for storing the tokenizer of distilgpt2

In [2]:
!pip install datasets

     |████████████████████████████████| 325 kB 8.3 MB/s 
     |████████████████████████████████| 136 kB 54.8 MB/s 
     |████████████████████████████████| 212 kB 53.3 MB/s 
     |████████████████████████████████| 1.1 MB 42.6 MB/s 
     |████████████████████████████████| 127 kB 71.6 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 271 kB 51.0 MB/s 
     |████████████████████████████████| 144 kB 71.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Now, its time for Training (or fine tuning) distilgpt2 with IMDB reviews
Given below is a command containing few parameters to help Transformers finetune distilgpt2. now, let's understand what these parameters mean

1) output_dir: It is the weights_dir we made where our finetuned model will be stored in the form of checkpoints

2) model_name_or_path: It tells the kind of model we are currently dealing with

3) per_device_train_batch_size: It tells the batch size for each gpu

4) do_train: It tells pytorch to start training mode

5) train_file: This is where we give the input text data 

6) num_train_epochs: Number of epochs for finetuning


Now, let the training begin...

In [3]:
weights_dir = "output"

In [8]:
cmd = '''
python transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path distilgpt2 \
    --train_file {0} \
    --do_train \
    --num_train_epochs 3 \
    --overwrite_output_dir \
    --per_device_train_batch_size 2 \
    --output_dir {1}
'''.format(file_name, weights_dir)

In [ ]:
!{cmd}

04/20/2022 03:22:15 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
04/20/2022 03:22:15 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TO

Although, Huggingface provides a run_generation.py file for language generation. Running it from a command (as it takes the input), makes it load the model and the tokenizer everytime you run the file which slows downs generation. To reduce the I/O overhead, I have restructured the run_generation.py file in the following code which only loads the model and tokenizer once in a model and a tokenizer object and we can use these objects to generate text over and over again

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def get_model_tokenizer(weights_dir, device = 'cuda'):
    print("Loading Model ...")
    model = GPT2LMHeadModel.from_pretrained(weights_dir)
    model.to('cuda')
    print("Model Loaded ...")
    tokenizer = GPT2Tokenizer.from_pretrained(weights_dir)
    return model, tokenizer

def generate_messages(
    model,
    tokenizer,
    prompt_text,
    stop_token,
    length,
    num_return_sequences,
    temperature = 0.7,
    k=20,
    p=0.9,
    repetition_penalty = 1.0,
    device = 'cuda'
):

    MAX_LENGTH = int(10000)
    def adjust_length_to_model(length, max_sequence_length):
        if length < 0 and max_sequence_length > 0:
            length = max_sequence_length
        elif 0 < max_sequence_length < length:
            length = max_sequence_length  # No generation bigger than model size
        elif length < 0:
            length = MAX_LENGTH  # avoid infinite loop
        return length
        
    length = adjust_length_to_model(length=length, max_sequence_length=model.config.max_position_embeddings)

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")

    encoded_prompt = encoded_prompt.to(device)

    output_sequences = model.generate(
            input_ids=encoded_prompt,
            max_length=length + len(encoded_prompt[0]),
            temperature=temperature,
            top_k=k,
            top_p=p,
            repetition_penalty=repetition_penalty,
            do_sample=True,
            num_return_sequences=num_return_sequences,
        )

    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequences = []

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        #print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()

        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # Remove all text after the stop token
        text = text[: text.find(stop_token) if stop_token else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        )

        generated_sequences.append(total_sequence)
    return generated_sequences


In [ ]:

model, tokenizer = get_model_tokenizer(weights_dir, device = 'cuda')


Loading Model ...
Model Loaded ...


In [ ]:
temperature = 1.0
k=400
p=0.9
repetition_penalty = 1.0
num_return_sequences = 5
length = 1000
stop_token = '|EndOfText|'
prompt_text = "this is"

In [ ]:
%%time
generate_messages(
    model,
    tokenizer,
    prompt_text,
    stop_token,
    length,
    num_return_sequences,
    temperature = temperature,
    k=k,
    p=p,
    repetition_penalty = repetition_penalty
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 9.84 s, sys: 1.85 s, total: 11.7 s
Wall time: 11.7 s


["this is a really great comic but it isn't. Its one of the best comics you'll ever see. I'll go buy it in the States and be honest, I'll not buy it as a mass sell. ",
 'this is a beautiful picture. All the different styles are very enjoyable. These were my only highlight.<br /><br />',
 'this is as important to us as anything else." ',
 "this is the best video of my life. Each scene has more to do with the story of its character and the humor it provides. For me, I'm looking forward to another look back. ",
 'this is to be handed back to the dead and must be stumped. The last time we saw "S" was in 1981. I might have missed it, but hey I know you guys are getting bored with most of the other \'artwork\' and have only read a few books - it\'s a waste of money. I actually like the concept, but it might make a new impression on some. It could have made a huge difference in the life of others, but I didn\'t really care - I\'d have given up on the chance anyway. So, I\'m glad I won\'t give